In [1]:
!pip install sumeval
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 896.1 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for sumeval: filename=sumeval-0.2.2-py3-none-any.whl size=54549 sha256=7b57bc3c9a25bf986ff8e404272355e6b78260efb651e829a8512bbd5ed42430
  Stored in directory: /root/.cache/pip/wheels/f4/3f/31/c521bdfba2be7518bd94ba3e8b982812822167cc0497fad192
Successfully built sumeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 737.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.2/379.2 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 9.8

# Decription 
### Fine-Tune a T5 model on the samsum dataset using Pytorch and HugingFace.

In [2]:
import gc
import random
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sumeval.metrics.rouge import RougeCalculator

import torch
from transformers import T5Tokenizer
import transformers
from transformers import T5ForConditionalGeneration
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import os
from datasets import load_dataset, Dataset
from transformers import get_scheduler
from torch.optim import AdamW
from torch.utils.data import DataLoader

print('Pytorch version: %s'  % torch.__version__)

Pytorch version: 1.11.0


In [3]:
warnings.simplefilter('ignore')
pd.set_option('display.max_colwidth', 100)

## Reading Data

In [4]:
dataset = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
def add_prefix(train, valid, test):
    # adding summarize in front of data
    train_df = pd.DataFrame(train)
    train_df["dialogue"] = "summarize: " + train_df["dialogue"]
    train = Dataset.from_pandas(train_df)

    valid_df = pd.DataFrame(valid)
    valid_df["dialogue"] = "summarize: " + valid_df["dialogue"]
    valid = Dataset.from_pandas(valid_df)

    test_df = pd.DataFrame(test)
    test_df["dialogue"] = "summarize: " + test_df["dialogue"]
    test = Dataset.from_pandas(test_df)

    return train, valid, test

def random_subset(train, frac):
    train_df = pd.DataFrame(train)
    train_df = train_df.sample(frac = frac, random_state = 2) 
    return Dataset.from_pandas(train_df)

In [6]:
train = dataset["train"]
valid = dataset["validation"]
test = dataset["test"]

train = train.remove_columns(["id"])
valid = valid.remove_columns(["id"])
test = test.remove_columns(["id"])

train, valid, test = add_prefix(train, valid, test)

print(len(train), len(valid), len(test))
print("dataset has features: ", train)
print("sample input and output is")
print(train[0]["dialogue"])
print(train[0]["summary"])


14732 818 819
dataset has features:  Dataset({
    features: ['dialogue', 'summary'],
    num_rows: 14732
})
sample input and output is
summarize: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
Amanda baked cookies and will bring Jerry some tomorrow.


In [7]:
# Taking 30% of train data for training
train = random_subset(train, 1)
print(len(train))

14732


## Building custom Data class

In [8]:
# this class object will be passed to the dataloader
class LoadData(torch.utils.data.Dataset):
    """
    Using this since dataloader expects map-style dataset objects
    
    """
    
    def __init__(
        self, dataset, tokenizer, source_length, target_length):
        """
        Initializes a Dataset class

        Args:
            dataset (Dataset object): Input Dataset
            tokenizer (Tokenizer object): Transformer tokenizer
            source_length (int): Max length of source text
            target_length (int): Max length of target text
        """
        
        self.tokenizer = tokenizer
        self.data = dataset
        self.source_length = source_length
        self.summary_length = target_length
        self.target_text = self.data["summary"]
        self.source_text = self.data["dialogue"]

    def __len__(self):
        return len(self.target_text)

    def __getitem__(self, index):
        """
        return input ids, attention masks and target ids
        
        """
        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.__call__(
            [source_text],
            max_length=self.source_length,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        
        target = self.tokenizer.__call__(
            [target_text],
            max_length=self.summary_length,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

In [9]:
rouge = RougeCalculator(stopwords=True, lang="en")

def rouge_calc(preds, targets):
    rouge_1 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=1) for i in range(len(preds))]
    rouge_2 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=2) for i in range(len(preds))]
    rouge_l = [rouge.rouge_l(summary=preds[i],references=targets[i]) for i in range(len(preds))]

    return {"Rouge_1": np.array(rouge_1).mean(),
            "Rouge_2": np.array(rouge_2).mean(),
            "Rouge_L": np.array(rouge_l).mean()}

def evaluate(model, eval_dataloader, tokenizer, device):
    predictions = []
    ground_truths = []
    losses = []
    
    with torch.no_grad():
        for eval_batch in eval_dataloader:
            y = eval_batch['target_ids'].to(device, dtype = torch.long)
            ids = eval_batch['source_ids'].to(device, dtype = torch.long)
            mask = eval_batch['source_mask'].to(device, dtype = torch.long)
            
            #print(type(ids), np.shape(ids))

            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              #num_beams=2,
              repetition_penalty=2.5,  # there is a research paper for this
              #length_penalty=1.0,  # > 0 encourages to generate short sentences, < 0 to generate long sentences
              #early_stopping=True  # stops beam search when number of beams sentences are generated per batch
              )
            
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]
            
            predictions += preds
            ground_truths += target
            
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100

            loss = model(
                input_ids=ids,
                attention_mask=mask,
                decoder_input_ids=y_ids,
                labels=lm_labels,
            )[0]
            losses.append(loss.item())

    scores = rouge_calc(predictions, ground_truths)
    avg_loss = sum(losses)/len(losses)
    print("Validation data score and losses are", scores, avg_loss)
    return avg_loss

def train_(model, train_loader, valid_loader, device, tokenizer, optimizer):
    steps = 0
    last_loss = 1000
    
    checkpoint_path = parameters["out_dir"] + "best_checkpoint/"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
    
    for epoch in range(parameters["epochs"]):
        print("Epoch: ", epoch)    
        for batch in train_loader:
            model.train()
            y = batch["target_ids"].to(device, dtype=torch.long)

            y_ids = y[:, :-1].contiguous()  # inputs passed to decoder, start token for decoder is pad token
            lm_labels = y[:, 1:].clone().detach()  # since it is lm task
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100  # so that loss can ignore the padded tokens

            ids = batch["source_ids"].to(device, dtype=torch.long)
            mask = batch["source_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                decoder_input_ids=y_ids,
                labels=lm_labels,
            )
            loss = outputs[0]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if steps % 400 == 0: print("Train loss on {}th step is {}".format(steps, loss.item()))
            """
            if steps % 400 == 0:
                model.eval()
                print("Train loss on {}th step is {}".format(steps, loss.item()))
                loss = evaluate(model, valid_loader, tokenizer, device)
                if loss < last_loss: # save model parameters
                    model.save_pretrained(checkpoint_path)
                    tokenizer.save_pretrained(checkpoint_path)
                    torch.save(optimizer.state_dict(), os.path.join(checkpoint_path, "optimizer.pt"))
                    last_loss = loss"""
            steps += 1
    
    """loss = evaluate(model, valid_loader, tokenizer)
    if loss < last_loss: # save model parameters
        model.save_pretrained(checkpoint_path)
        tokenizer.save_pretrained(checkpoint_path)
        torch.save(optimizer.state_dict(), os.path.join(checkpoint_path, "optimizer.pt"))
        last_loss = loss"""
    
    # save the last model weights
    model.save_pretrained(parameters["out_dir"])
    tokenizer.save_pretrained(parameters["out_dir"])
    torch.save(optimizer.state_dict(), os.path.join(parameters["out_dir"], "optimizer.pt"))

def train_model(parameters, train_dataset, valid_dataset):
    cuda =  torch.cuda.is_available()
    device = torch.device("cuda") if cuda else torch.device("cpu")
    
    tokenizer = T5Tokenizer.from_pretrained(parameters["model"])    
    model = T5ForConditionalGeneration.from_pretrained(parameters["model"])  # has 60M parameters
    model = model.to(device)
    
    optimizer = AdamW(model.parameters(), lr=parameters["lr"], weight_decay=parameters["wd"])
    
    train_obj = LoadData(
        train_dataset,
        tokenizer,
        parameters["max_source_length"],
        parameters["max_target_length"]
    )
    
    val_obj = LoadData(
        valid_dataset,
        tokenizer,
        parameters["max_source_length"],
        parameters["max_target_length"]
    )
    
    train_loader = DataLoader(train_obj, shuffle=True, batch_size=parameters["train_bs"])
    valid_loader = DataLoader(val_obj, shuffle=False, batch_size=parameters["val_bs"])
    
    num_training_steps = parameters["epochs"] * len(train_loader)
    print("Training steps are", num_training_steps)

    train_(model, train_loader, valid_loader, device, tokenizer, optimizer)

In [14]:
parameters = {"model": "t5-small",  # model_type: t5-base/t5-large
    "train_bs": 8,  # training batch size
    "val_bs": 8,  # validation batch size
    "epochs": 5,  # number of training epochs
    "lr": 1e-4,  # learning rate
    "wd": 0.01,  # learning rate
    "max_source_length": 512,  # max length of source text
    "max_target_length": 80,  # max length of target text
    "SEED": 42,
    "out_dir": "./T5-samsum-alltrain/"}

In [13]:
train_model(parameters, train, valid)

Training steps are 8290
Epoch:  0
Train loss on 0th step is 4.062132835388184
Train loss on 400th step is 1.9140644073486328
Train loss on 800th step is 1.9817224740982056
Train loss on 1200th step is 2.3413779735565186
Train loss on 1600th step is 1.4699466228485107
Epoch:  1
Train loss on 2000th step is 1.9198647737503052
Train loss on 2400th step is 1.8585165739059448
Train loss on 2800th step is 1.9449955224990845
Train loss on 3200th step is 1.4649343490600586
Epoch:  2
Train loss on 3600th step is 1.997383713722229
Train loss on 4000th step is 1.1843547821044922
Train loss on 4400th step is 2.2455718517303467
Train loss on 4800th step is 1.4643442630767822
Epoch:  3
Train loss on 5200th step is 1.4191169738769531
Train loss on 5600th step is 1.9597364664077759
Train loss on 6000th step is 1.8833582401275635
Train loss on 6400th step is 1.117913842201233
Epoch:  4
Train loss on 6800th step is 1.7629506587982178
Train loss on 7200th step is 2.04990291595459
Train loss on 7600th ste

## Make some predictions

In [15]:
cuda =  torch.cuda.is_available()
device = torch.device("cuda") if cuda else torch.device("cpu")

tokenizer = T5Tokenizer.from_pretrained(parameters["out_dir"], do_lower_case=False)
model = T5ForConditionalGeneration.from_pretrained(parameters["out_dir"])
model.to(device)

val_obj = LoadData(
        valid,
        tokenizer,
        parameters["max_source_length"],
        parameters["max_target_length"]
    )
valid_loader = DataLoader(val_obj, shuffle=False, batch_size=parameters["val_bs"])

test_obj = LoadData(
        test,
        tokenizer,
        parameters["max_source_length"],
        parameters["max_target_length"]
    )

test_loader = DataLoader(test_obj, shuffle=False, batch_size=parameters["val_bs"])

evaluate(model, valid_loader, tokenizer, device)
evaluate(model, test_loader, tokenizer, device)

Validation data score and losses are {'Rouge_1': 0.5005497255067427, 'Rouge_2': 0.18086332729221619, 'Rouge_L': 0.41342872683905424} 1.701273875329101
Validation data score and losses are {'Rouge_1': 0.49109948828069394, 'Rouge_2': 0.17406329182965535, 'Rouge_L': 0.4128331616732573} 1.7074786725553495


1.7074786725553495

In [12]:
from IPython.display import FileLink

FileLink(r'pytorch_model.bin')

/kaggle/working/pytorch_model.bin

In [16]:
# load the pre-trained best-checkpoint model
def inference(model, device, tokenizer, parameters):
    dataset = load_dataset("samsum")
    test = dataset["test"]
    test = test.remove_columns(["id"])
    
    sent = test[0]["dialogue"]  # taking an example sentence
    sent = " ".join(sent.split())
    
    source = tokenizer.__call__(
            [sent],
            max_length=parameters["max_source_length"],
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
    
    ids = source["input_ids"]
    mask = source["attention_mask"]
    
    model.eval()
    with torch.no_grad():
        ids = ids.to(device, dtype = torch.long)
        mask = mask.to(device, dtype = torch.long)
        
        generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              #num_beams=2,
              repetition_penalty=2.5,  # there is a research paper for this
              #length_penalty=1.0,  # > 0 encourages to generate short sentences, < 0 to generate long sentences
              #early_stopping=True  # stops beam search when number of beams sentences are generated per batch
              )
        
        preds = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        
        print("Input dialogue is: ", sent)
        print("Output summary is: ", preds)
    
    
    
cuda =  torch.cuda.is_available()
device = torch.device("cuda") if cuda else torch.device("cpu")

tokenizer = T5Tokenizer.from_pretrained(parameters["out_dir"], do_lower_case=False)
model = T5ForConditionalGeneration.from_pretrained(parameters["out_dir"])
model.to(device)
inference(model, device, tokenizer, parameters)

  0%|          | 0/3 [00:00<?, ?it/s]

Input dialogue is:  Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: <file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: <file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him 🙂 Hannah: Urgh.. Alright Hannah: Bye Amanda: Bye bye
Output summary is:  Hannah has Betty's number. Amanda can't find it. Larry called her last time they were at the park together.


In [17]:
# save the results in a file
outputs = {"source_text": [], "prediction": [], "label": [], "split": []}

model.eval()

with torch.no_grad():
    for eval_batch in valid_loader:
        y = eval_batch['target_ids'].to(device, dtype = torch.long)
        ids = eval_batch['source_ids'].to(device, dtype = torch.long)
        mask = eval_batch['source_mask'].to(device, dtype = torch.long)

        generated_ids = model.generate(
          input_ids = ids,
          attention_mask = mask, 
          max_length=150, 
          #num_beams=2,
          repetition_penalty=2.5,  # there is a research paper for this
          #length_penalty=1.0,  # > 0 encourages to generate short sentences, < 0 to generate long sentences
          #early_stopping=True  # stops beam search when number of beams sentences are generated per batch
          )

        preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
        sources = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in ids]
        target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]
        
        for i,out in enumerate(preds):
            outputs["prediction"].append(out)
            outputs["label"].append(target[i])
            outputs["source_text"].append(sources[i])
            outputs["split"].append("valid")
    
    for test_batch in test_loader:
        y = test_batch['target_ids'].to(device, dtype = torch.long)
        ids = test_batch['source_ids'].to(device, dtype = torch.long)
        mask = test_batch['source_mask'].to(device, dtype = torch.long)

        generated_ids = model.generate(
          input_ids = ids,
          attention_mask = mask, 
          max_length=150, 
          #num_beams=2,
          repetition_penalty=2.5,  # there is a research paper for this
          #length_penalty=1.0,  # > 0 encourages to generate short sentences, < 0 to generate long sentences
          #early_stopping=True  # stops beam search when number of beams sentences are generated per batch
          )

        preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
        sources = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in ids]
        target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]
        
        for i,out in enumerate(preds):
            outputs["prediction"].append(out)
            outputs["label"].append(target[i])
            outputs["source_text"].append(sources[i])
            outputs["split"].append("test")


outputs = pd.DataFrame.from_dict(outputs)
outputs.to_csv("./T5_samsumall.csv")